In [ ]:
# @title
import os
import subprocess
import sys
import fileinput
import time
from datetime import datetime
import requests
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
from PIL import Image
from IPython.display import display, HTML, Image
from io import BytesIO
import json
import subprocess
import threading
import time
import socket
import urllib.request

def cloudflare_thread(port):
  import socket
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("Link sever sử dụng Online:", l[l.find("http"):], end='')

def localtunnel_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nKhởi chạy localtunnel Sever\n")

  print("Mật khẩu cho localtunnel là:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

def sever_flare(port):
  threading.Thread(target=localtunnel_thread, daemon=True, args=(port,)).start()
  threading.Thread(target=cloudflare_thread, daemon=True, args=(port,)).start()

def inf(msg, style, wdth):
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)
def aria_down(link,path,name):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d  {path} -o {name}

def c_folder(path):
  path = f'{User_folder}/{path}'
  os.makedirs(path, exist_ok=True)

def default_file_check(folder_source,folder_check,over=False):
  for item in os.listdir(folder_source):
    file_path = os.path.join(folder_source, item)
    if os.path.isfile(file_path):
      cp_path = file_path.replace(SDVNFolder, folder_check)
      !cp {'-f' if over else '-n'} {file_path} {cp_path}   
    elif os.path.isdir(file_path):
      default_file_check(file_path,folder_check,over)

def run_list_txt(txt):
  with open(f"{txt}", "r") as file:
    lists = file.readlines()
  for list in lists :
    !{list}

def link_folder(source,target):
  !rm -r {target}
  !ln -s {source} {target}

def split_string_at(link):
    delimiter = "@="
    if delimiter in link:
        parts = link.split(delimiter)
        return parts[0], f'{parts[1]}.safetensors'
    else:
        return link, None

def download(link):
  link = link.replace('&', '\&')
  valuelink, valuename = split_string_at(link)
  if '#' not in link:
    if valuename != None :
      aria_down(valuelink,checkpointpatch,valuename)
    else :
      !wget {"" if "?token=" in valuelink else "-c"} {valuelink} -P {checkpointpatch} --content-disposition

def download_lib(model):
  downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
  if 'https:' in model :
    download(model)
  elif '.ckpt' not in model and '.gguf' not in model:
    model += '.safetensors'
    !{downloadcode} {modellist[model]} -d {checkpointpatch} -o {model}

def download_txt(lines):
  for line in lines:
    download(line)

def run_ver(Version,CommandLine):
  %cd {Version_folder}
  if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2":
    sever_flare(7860)
    drive_arg = f"--ckpt-dir {User_folder}/Model --embeddings-dir {User_folder}/Embeddings" if DriveLib else ""
    final_arg = f"--xformers --theme dark --enable-insecure-extension-access --disable-safe-unpickle --share --no-half-vae {CommandLine} \
                --ui-settings-file {User_folder}/Setting/WebUI_Setting.json \
                --styles-file {User_folder}/Setting/WebUI_Styles.csv --lora-dir {User_folder}/Lora {drive_arg}" 
    !python launch.py {final_arg}
  if Version == "ComfyUI":
    sever_flare(8888)
    final_arg = f"--preview-method auto --port 8888 --input-directory {User_folder}/ComfyUIinput {CommandLine}" 
    !python main.py {final_arg}
  if Version == "Kohya" :
    !python {Version_folder}/kohya_gui.py --share --headless
  if Version == "Fooocus" :
    !python {Version_folder}/entry_with_update.py --share --theme dark --disable-preset-download {CommandLine}
  if Version == "AutoRetouch":
    !python {Version_folder}/auto_retoucher.py

%cd {root_folder}
folder_list = [ '',
                'Model',
                'Lora',
                'Embeddings',
                'wildcards',
                'Hypernetworks',
                'Export',
                'Export/ComfyUI',
                'Export/Fooocus',
                'Export/AutoRetouch',
                'ControlnetModel',
                'TrainData',
                'AnimateDiffModel',
                'Lora/AnimateDiffLora',
                'ComfyUIinput',
                'Setting',
                'Setting/Comfy_Setting',
                'Setting/Comfy_Setting/default',
                'Setting/Comfy_Setting/default/workflows']
for i in folder_list:
  c_folder(i)

default_file_check(f"{SDVNFolder}/Setting",User_folder)

file_path = f'{User_folder}/Setting/WebUI_Setting.json'
with open(file_path, 'r') as file:
    json_str = file.read()
default_path = "/content/drive/MyDrive/SD-Data"
user_path = f"{User_folder}"
json_str = json_str.replace(default_path, user_path)
with open(file_path, 'w') as file:
    file.write(json_str)
clear_output()

html="<h1 style='color:lightgreen; font-size:3em' ><code>BẮT ĐẦU CÀI ĐẶT CHƯƠNG TRÌNH - VUI LÒNG ĐỢI ...</code></h1><h3><code>☕️ Pha một ly cà phê đợi 5-10 phút nữa nhé ...</code></h3><h3><code>☕️ Cài tuỳ chọn Extension/Note/ thông qua thẻ txt tương ứng trong thư mục Setting trong thư mục SD-Data</code></h3><p>_________</p>"
display(HTML(html))
image_url = 'https://github.com/StableDiffusionVN/SDVN-WebUI/blob/main/huongdan.png?raw=true'
response = requests.get(image_url)
display(Image(data=response.content))

if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2" or Version == "ComfyUI":
  if Controlnet != "none":
    if Prune:
      !git clone https://huggingface.co/StableDiffusionVN/Prunedcontrolnet {SDVNFolder}/ControlNet/Basecontrolnet
    else:
      !git clone https://huggingface.co/StableDiffusionVN/Basecontrolnet {SDVNFolder}/ControlNet/Basecontrolnet
  if Controlnet == "extra" :
    !git clone https://huggingface.co/StableDiffusionVN/Extracontrolnet {SDVNFolder}/ControlNet/Extracontrolnet
  if Controlnet == "XL" :
    !git clone https://huggingface.co/StableDiffusionVN/XLControlnet {SDVNFolder}/ControlNet/XLControlnet
  !ln -s {User_folder}/ControlnetModel {SDVNFolder}/ControlNet
  link_folder(f'{User_folder}/ControlnetModel',f'{SDVNFolder}/ControlNet/ControlnetModel')
  !git clone https://github.com/StableDiffusionVN/WC-SDVN
  link_folder(f'{root_folder}/WC-SDVN',f'{User_folder}/wildcards/WC-SDVN')
